# Load and preprocess 2004 data

We will, over time, look over other years. Our current goal is to explore the features of a single year.

---

In [1]:
%pylab --no-import-all inline
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


## Load the data.

---

If this fails, be sure that you've saved your own data in the prescribed location, then retry.

In [11]:
file = "../data/interim/2004data.dta"
df_rawest = pd.read_stata(file)

In [12]:
good_columns = [
    'V043116',  # Party Identification
    
    'V045132',  #Abortion
    'V045189',  #Moral Relativism
    'V045190',  #“Newer Lifestyles”
    'V045191',  #Moral Tolerance
    'V045192',  #Traditional Families
    'V045156a',  #Gay Job Discrimination
    'V045158',  # Gay adoption
    
    'V043150',  #National Health Insurance
    'V043152',  #Guaranteed Job
    'V045121',  #Services/Spending
    
    'V045207a',  #Affirmative Action
    'V045193',  #Racial Resentment 1
    'V045194',  #Racial Resentment2
    'V045195',  #Racial Resentment3
    'V045196',  #Racial Resentment4
]
df_raw = df_rawest[good_columns]

## Clean the data
---

In [20]:
def convert_to_int(s):
    """Turn ANES data entry into an integer.
    
    >>> convert_to_int("1. Govt should provide many fewer services")
    1
    >>> convert_to_int("2")
    2
    """
    try:
        return int(s.partition('.')[0])
    except ValueError:
        warnings.warn("Couldn't convert: "+s)
        return np.nan
    except AttributeError:
        return s
    
def not_informative_to_nan(x):
    """Convert non-informative values to missing.
    
    ANES codes various non-answers as 8, 9, and 0.
    For instance, if a question does not pertain to the 
    respondent.
    """
    return np.nan if x in {8, 9, 0} else x

df = df_raw.applymap(convert_to_int)
non_pid_columns = list(df.columns)
non_pid_columns.remove('V043116')
non_pid_columns.remove('V045132')
df[non_pid_columns] = df[non_pid_columns].applymap(not_informative_to_nan)

df.loc[:, 'V045132'] = df.V045132.apply(lambda x: np.nan if x in {7, 8, 9, 0} else x)  # Abortion is 1-4 and 7 is 'other'


# Code so that liberal is higher numbers
df.loc[:, 'V043116'] = df.V043116.apply(lambda x: np.nan if x >= 7 else -x)  # 7: other minor party, 8: apolitical, 9: NA

df.loc[:, 'V045189'] = df.V045189.apply(lambda x: -x)  # 1: moral relativism, 5: no relativism
df.loc[:, 'V045191'] = df.V045191.apply(lambda x: -x)  # Tolerance. 1: tolerance, 7: not
df.loc[:, 'V045156a'] = df.V045156a.apply(lambda x: -x)  # protect gays against discrimination
df.loc[:, 'V045158'] = df.V045158.apply(lambda x: -x)  # support gay adoption


df.loc[:, 'V043150'] = df.V043150.apply(lambda x: -x)  # Gov't insurance?
df.loc[:, 'V043152'] = df.V043152.apply(lambda x: -x)  # Gov't guaranteed job?

df.loc[:, 'V045207a'] = df.V045207a.apply(lambda x: -x)  # affirmative action
df.loc[:, 'V045194'] = df.V045194.apply(lambda x: -x)  # Racial support
df.loc[:, 'V045195'] = df.V045195.apply(lambda x: -x)  # Systemic factors?

df.rename(inplace=True, columns=dict(zip(
    good_columns,
    ["PartyID",
    
    "Abortion",
    "MoralRelativism",
    "NewerLifestyles",
    "MoralTolerance",
    "TraditionalFamilies",
    "GayJobDiscrimination",
    "GayAdoption",

    "NationalHealthInsurance",
    "StandardOfLiving",
    "ServicesVsSpending",

    "AffirmativeAction",
    "RacialResentment1",
    "RacialResentment2",
    "RacialResentment3",
    "RacialResentment4",
    ]
)))

In [21]:
print("Variables now available: df")

Variables now available: df


In [22]:
df_rawest.V043116.value_counts()

2. Independent-Democrat (3-4-5/./5)        210
0. Strong Democrat (2/1/.)                 203
6. Strong Republican (1/1/.)               193
1. Weak Democrat (2/5-8-9/.)               179
5. Weak Republican (1/5-8-9/.)             154
4. Independent-Republican (3-4-5/./1)      138
3. Independent-Independent                 118
9. DK (8/./.)                                8
7. Other; minor party; refuses to say        5
8. Apolitical (5/./3-8-9 if apolitical)      4
Name: V043116, dtype: int64

In [24]:
df.PartyID.value_counts()

-2.0    210
 0.0    203
-6.0    193
-1.0    179
-5.0    154
-4.0    138
-3.0    118
Name: PartyID, dtype: int64

In [25]:
df.head()

,PartyID,Abortion,MoralRelativism,NewerLifestyles,MoralTolerance,TraditionalFamilies,GayJobDiscrimination,GayAdoption,NationalHealthInsurance,StandardOfLiving,ServicesVsSpending,AffirmativeAction,RacialResentment1,RacialResentment2,RacialResentment3,RacialResentment4
0,-3.0,2.0,-2.0,3.0,-2.0,1.0,-1.0,-1.0,-7.0,-7.0,4.0,-5.0,1.0,-5.0,-2.0,1.0
1,-2.0,2.0,-2.0,2.0,-2.0,1.0,-1.0,-1.0,-2.0,-3.0,3.0,-4.0,2.0,-1.0,-2.0,2.0
2,-6.0,2.0,-5.0,2.0,-2.0,2.0,-2.0,-1.0,-4.0,-4.0,3.0,-4.0,2.0,-4.0,-3.0,2.0
3,-3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,-6.0,2.0,-5.0,1.0,-3.0,1.0,-5.0,-5.0,-5.0,-6.0,4.0,-5.0,1.0,-5.0,-2.0,2.0


In [26]:
df.to_csv("../data/processed/2004.csv")